In [ ]:
# Import các thư viện cần thiết
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib
import matplotlib.pyplot as plt

# Đọc dữ liệu
data = pd.read_csv('clean_data.csv')

# Xử lý dữ liệu: Chọn các cột dùng để dự đoán, loại bỏ cột không cần thiết
X = data.drop(['Weekly_Sales', 'Date'], axis=1)  # Bỏ cột Weekly_Sales và Date
y = data['Weekly_Sales']  # Biến phụ thuộc

# Định nghĩa các cột
numeric_features = X.select_dtypes(include=['float64', 'int64']).columns
categorical_features = ['IsHoliday', 'Type']  # Các cột phân loại

# Sử dụng ColumnTransformer để xử lý đồng thời
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(drop='first'), categorical_features)
    ])

# Chia dữ liệu thành tập train và test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Chuyển đổi dữ liệu
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

# Danh sách các mô hình dự đoán
models = {
    'LinearRegression': LinearRegression(),
    'RandomForest': RandomForestRegressor(n_jobs=-1),  # Sử dụng tất cả các lõi CPU
    'SVR': SVR(),
    'KNeighbors': KNeighborsRegressor(),
    'XGBoost': XGBRegressor(n_jobs=-1)  # Sử dụng tất cả các lõi CPU
}

# Dictionary để lưu các kết quả đánh giá
results = {}

# Huấn luyện và lưu các mô hình
for name, model in models.items():
    # Huấn luyện mô hình
    model.fit(X_train_transformed, y_train)
    
    # Dự đoán trên tập test
    y_pred = model.predict(X_test_transformed)
    
    # Tính toán các chỉ số đánh giá
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Lưu kết quả
    results[name] = {'MSE': mse, 'R2_Score': r2}
    
    # Lưu mô hình với joblib
    joblib.dump(model, f'{name}_model.pkl')

# Hiển thị kết quả đánh giá
for name, metrics in results.items():
    print(f"{name}: MSE = {metrics['MSE']}, R2 Score = {metrics['R2_Score']}")

# Lưu preprocessor để sử dụng lại sau
joblib.dump(preprocessor, 'preprocessor.pkl')

# Tạo biểu đồ thanh so sánh MSE và R2 Score
mse_values = [metrics['MSE'] for metrics in results.values()]
r2_values = [metrics['R2_Score'] for metrics in results.values()]
model_names = list(results.keys())

# Tạo biểu đồ MSE
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.bar(model_names, mse_values, color='skyblue')
plt.title('MSE Comparison')
plt.ylabel('Mean Squared Error')

# Tạo biểu đồ R2 Score
plt.subplot(1, 2, 2)
plt.bar(model_names, r2_values, color='lightgreen')
plt.title('R2 Score Comparison')
plt.ylabel('R2 Score')

# Hiển thị biểu đồ
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import joblib

# Bước 1: Tạo dữ liệu giả lập
data = {
    'Store': [1, 2, 3, 4, 5, 6, 7],
    'Dept': [1, 2, 3, 4, 5, 1, 2],
    'Date': pd.to_datetime(['2024-01-03', '2024-02-15', '2024-03-10', '2024-11-22', 
                             '2024-12-25', '2024-07-04', '2024-05-10']),
    'IsHoliday': ['Normal Day', 'Super Bowl', 'Labor Day', 'Thanksgiving', 
                  'Christmas', 'Normal Day', 'Normal Day'],
    'Temperature': [45.2, 55.1, 60.3, 48.5, 32.0, 78.5, 85.3],
    'Fuel_Price': [2.85, 3.02, 3.25, 2.95, 2.65, 3.10, 2.90],
    'MarkDown1': [1500.0, 0.0, 2500.0, 1800.0, 3000.0, 500.0, 350.0],
    'MarkDown2': [800.0, 0.0, 1500.0, 1000.0, 2500.0, 0.0, 150.0],
    'MarkDown3': [0.0, 0.0, 800.0, 0.0, 1200.0, 0.0, 0.0],
    'MarkDown4': [400.0, 0.0, 0.0, 300.0, 500.0, 200.0, 0.0],
    'MarkDown5': [500.0, 0.0, 0.0, 0.0, 400.0, 100.0, 0.0],
    'CPI': [211.10, 221.50, 211.85, 220.10, 210.20, 230.55, 215.80],
    'Unemployment': [7.2, 6.8, 8.0, 6.9, 7.5, 6.0, 7.1],
    'Type': ['A', 'B', 'A', 'C', 'A', 'B', 'C'],
    'Size': [151000, 202000, 151000, 103000, 151000, 202000, 103000]
}

# Tạo DataFrame từ dữ liệu
new_data = pd.DataFrame(data)

# Bước 2: Xử lý dữ liệu
new_data_processed = pd.get_dummies(new_data, columns=['IsHoliday', 'Type'], drop_first=True)

# Load scaler đã lưu để chuẩn hóa dữ liệu
scaler = joblib.load('scaler.pkl')
new_data_scaled = scaler.transform(new_data_processed)

# Bước 3: Dự đoán với các mô hình đã lưu
models = ['LinearRegression', 'RandomForest', 'SVR', 'KNeighbors', 'XGBoost']
predictions = {}

for model_name in models:
    # Load mô hình từ file
    model = joblib.load(f'{model_name}_model.pkl')
    
    # Thực hiện dự đoán
    predictions[model_name] = model.predict(new_data_scaled)

# Bước 4: Hiển thị kết quả dự đoán
for model_name, preds in predictions.items():
    print(f'Predictions using {model_name}: {preds}')
